In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import time, datetime
import warnings
warnings.filterwarnings("ignore") 
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm
from ipywidgets import FloatProgress

In [47]:
df=pd.read_csv("E:\Projects\Product Recommendation\Amazon\Amazon.csv")

In [48]:
df.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹399,"₹1,099",64%,4.2,"24,269","High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7...","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQ...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original i...",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
1,B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZA...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RYGGS0M09S3KY,R17KQRUTAN5DKS,R3AAQGS6HP2QUK,R1HDNOG6TO2CCA,R3PHKXYA5AFEOU","A Good Braided Cable for Your Type C Device,Good quality product from ambrane,Super cable,As,Good quality,Good product,its good,Good quality for the price but one issue with my unit",I ordered this cable to connect my phone to Android Auto of car. The cable is really strong and the connection ports are really well made. I already has a Micro USB cable from Ambrane and it's sti...,https://m.media-amazon.com/images/W/WEBP_402378-T2/images/I/31zOsqQOAOL._SY445_SX342_QL70_FMwebp_.jpg,https://www.amazon.in/Ambrane-Unbreakable-Charging-Braided-Cable/dp/B098NS6PVG/ref=sr_1_2?qid=1672909124&s=electronics&sr=1-2
2,B096MSW6CT,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safety proctections and four-core copper wires promote maximum signal quality and strength and enhance charging & data transfer speed with up to 480 mb/s t...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQLWQOWZ4N3OA,AHTPQRIMGUD4BYR5YIHBH3CCGEFQ,AEUVWXYP5LT7PZLLZENEO2NODPBQ,AHC7MPW55DOO6WNCOQVA2VHOD26A,AFDI6FRPFBTNBG7BAEB7JDJSMKDQ,AFQKCEEEKXCOHTDG4WUN3X...","Kunal,Himanshu,viswanath,sai niharka,saqib malik,Aashiq,Ramu Challa,Sanjay gupta","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R25X4TBMPY91LX,R27OK7G99VK

In [49]:
pd.set_option('display.width', 1000)

In [50]:
pd.set_option('max_colwidth', 200)
df['product_link'].head()

0           https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
1       https://www.amazon.in/Ambrane-Unbreakable-Charging-Braided-Cable/dp/B098NS6PVG/ref=sr_1_2?qid=1672909124&s=electronics&sr=1-2
2        https://www.amazon.in/Sounce-iPhone-Charging-Compatible-Devices/dp/B096MSW6CT/ref=sr_1_3?qid=1672909124&s=electronics&sr=1-3
3     https://www.amazon.in/Deuce-300-Resistant-Tangle-Free-Transmission/dp/B08HDJ86NZ/ref=sr_1_4?qid=1672909124&s=electronics&sr=1-4
4    https://www.amazon.in/Portronics-Konnect-POR-1080-Charging-Function/dp/B08CF3B7N1/ref=sr_1_5?qid=1672909124&s=electronics&sr=1-5
Name: product_link, dtype: object

In [51]:
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [52]:
df.duplicated().sum()

0

In [53]:
df1 =df[['product_id','product_name','category','about_product']]

In [54]:
df1['product_name_tag']=df1['product_name'].apply(lambda x:x.split())
df1['category']=df1['category'].apply(lambda x:x.split("|"))
df1['about_product']=df1['about_product'].apply(lambda x:x.split())

In [55]:
df1.head()

,product_id,product_name,category,about_product,product_name_tag
0,B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables]","[High, Compatibility, :, Compatible, With, iPhone, 12,, 11,, X/XsMax/Xr, ,iPhone, 8/8, Plus,iPhone, 7/7, Plus,iPhone, 6s/6s, Plus,iPhone, 6/6, Plus,iPhone, 5/5s/5c/se,iPad, Pro,iPad, Air, 1/2,iPad...","[Wayona, Nylon, Braided, USB, to, Lightning, Fast, Charging, and, Data, Sync, Cable, Compatible, for, iPhone, 13,, 12,11,, X,, 8,, 7,, 6,, 5,, iPad, Air,, Pro,, Mini, (3, FT, Pack, of, 1,, Grey)]"
1,B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables]","[Compatible, with, all, Type, C, enabled, devices,, be, it, an, android, smartphone, (Mi,, Samsung,, Oppo,, Vivo,, Realme,, OnePlus,, etc),, tablet,, laptop, (Macbook,, Chromebook,, etc)|Supports,...","[Ambrane, Unbreakable, 60W, /, 3A, Fast, Charging, 1.5m, Braided, Type, C, Cable, for, Smartphones,, Tablets,, Laptops, &, other, Type, C, devices,, PD, Technology,, 480Mbps, Data, Sync,, Quick, C..."
2,B096MSW6CT,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables]","[【, Fast, Charger&, Data, Sync】-With, built-in, safety, proctections, and, four-core, copper, wires, promote, maximum, signal, quality, and, strength, and, enhance, charging, &, data, transfer, sp...","[Sounce, Fast, Phone, Charging, Cable, &, Data, Sync, USB, Cable, Compatible, for, iPhone, 13,, 12,11,, X,, 8,, 7,, 6,, 5,, iPad, Air,, Pro,, Mini, &, iOS, Devices]"
3,B08HDJ86NZ,"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables]","[The, boAt, Deuce, USB, 300, 2, in, 1, cable, is, compatible, with, smartphones,, tablets,, PC, peripherals,, Bluetooth, speakers,, power, banks, and, all, other, devices, with, Type-C, as, well, ...","[boAt, Deuce, USB, 300, 2, in, 1, Type-C, &, Micro, USB, Stress, Resistant,, Tangle-Free,, Sturdy, Cable, with, 3A, Fast, Charging, &, 480mbps, Data, Transmission,, 10000+, Bends, Lifespan, and, E..."
4,B08CF3B7N1,"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables]","[[CHARGE, &, SYNC, FUNCTION]-, This, cable, comes, with, charging, &, Data, sync, function|[HIGH, QUALITY, MATERIAL]-, TPE, +, Nylon, Material, to, make, sure, that, the, life, of, the, cable, is,...","[Portronics, Konnect, L, 1.2M, Fast, Charging, 3A, 8, Pin, USB, Cable, with, Charge, &, Sync, Function, for, iPhone,, iPad, (Grey)]"


In [56]:
df1['tags'] =  df1['category'] + df1['about_product']

In [57]:
new_df=df1[['product_id','product_name','tags']]
new_df.head()

,product_id,product_name,tags
0,B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables, High, Compatibility, :, Compatible, With, iPhone, 12,, 11,, X/XsMax/Xr, ,iPhone, 8/8, Plus,iPhone, 7/7, Plus..."
1,B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables, Compatible, with, all, Type, C, enabled, devices,, be, it, an, android, smartphone, (Mi,, Samsung,, Oppo,, V..."
2,B096MSW6CT,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables, 【, Fast, Charger&, Data, Sync】-With, built-in, safety, proctections, and, four-core, copper, wires, promote,..."
3,B08HDJ86NZ,"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables, The, boAt, Deuce, USB, 300, 2, in, 1, cable, is, compatible, with, smartphones,, tablets,, PC, peripherals,,..."
4,B08CF3B7N1,"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)","[Computers&Accessories, Accessories&Peripherals, Cables&Accessories, Cables, USBCables, [CHARGE, &, SYNC, FUNCTION]-, This, cable, comes, with, charging, &, Data, sync, function|[HIGH, QUALITY, MA..."


In [58]:
new_df['tags'] = new_df['tags'].apply(lambda x:"".join(x))

In [59]:
new_df.head()

,product_id,product_name,tags
0,B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)","Computers&AccessoriesAccessories&PeripheralsCables&AccessoriesCablesUSBCablesHighCompatibility:CompatibleWithiPhone12,11,X/XsMax/Xr,iPhone8/8Plus,iPhone7/7Plus,iPhone6s/6sPlus,iPhone6/6Plus,iPhone..."
1,B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)","Computers&AccessoriesAccessories&PeripheralsCables&AccessoriesCablesUSBCablesCompatiblewithallTypeCenableddevices,beitanandroidsmartphone(Mi,Samsung,Oppo,Vivo,Realme,OnePlus,etc),tablet,laptop(Mac..."
2,B096MSW6CT,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",Computers&AccessoriesAccessories&PeripheralsCables&AccessoriesCablesUSBCables【FastCharger&DataSync】-Withbuilt-insafetyproctectionsandfour-corecopperwirespromotemaximumsignalqualityandstrengthanden...
3,B08HDJ86NZ,"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)","Computers&AccessoriesAccessories&PeripheralsCables&AccessoriesCablesUSBCablesTheboAtDeuceUSB3002in1cableiscompatiblewithsmartphones,tablets,PCperipherals,Bluetoothspeakers,powerbanksandallotherdev..."
4,B08CF3B7N1,"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)",Computers&AccessoriesAccessories&PeripheralsCables&AccessoriesCablesUSBCables[CHARGE&SYNCFUNCTION]-Thiscablecomeswithcharging&Datasyncfunction|[HIGHQUALITYMATERIAL]-TPE+NylonMaterialtomakesurethat...


In [60]:
ps = PorterStemmer()
def stem(txt):
    y= []
    for i in txt.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [61]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [62]:
new_df.head()

,product_id,product_name,tags
0,B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)","computers&accessoriesaccessories&peripheralscables&accessoriescablesusbcableshighcompatibility:compatiblewithiphone12,11,x/xsmax/xr,iphone8/8plus,iphone7/7plus,iphone6s/6splus,iphone6/6plus,iphone..."
1,B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)","computers&accessoriesaccessories&peripheralscables&accessoriescablesusbcablescompatiblewithalltypecenableddevices,beitanandroidsmartphone(mi,samsung,oppo,vivo,realme,oneplus,etc),tablet,laptop(mac..."
2,B096MSW6CT,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",computers&accessoriesaccessories&peripheralscables&accessoriescablesusbcables【fastcharger&datasync】-withbuilt-insafetyproctectionsandfour-corecopperwirespromotemaximumsignalqualityandstrengthanden...
3,B08HDJ86NZ,"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)","computers&accessoriesaccessories&peripheralscables&accessoriescablesusbcablestheboatdeuceusb3002in1cableiscompatiblewithsmartphones,tablets,pcperipherals,bluetoothspeakers,powerbanksandallotherdev..."
4,B08CF3B7N1,"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)",computers&accessoriesaccessories&peripheralscables&accessoriescablesusbcables[charge&syncfunction]-thiscablecomeswithcharging&datasyncfunction|[highqualitymaterial]-tpe+nylonmaterialtomakesurethat...


In [63]:
cv = CountVectorizer(max_features=5000,stop_words='english',encoding='utf-8')

In [64]:
vectors = cv.fit_transform(new_df['tags'].values.astype(str))
vectors.shape

(1465, 5000)

In [65]:
cv.get_feature_names_out()

array(['000',
       '000bendingandinsertiontestsforextraprotectionanddurability',
       '000bendingtestsandcaneasilywithstanddailyuse', ..., 'zee5andmore',
       'zeolite', 'zoom'], dtype=object)

In [66]:
similar = cosine_similarity(vectors)

In [67]:
similar

array([[1.        , 0.08581944, 0.12155416, ..., 0.        , 0.        ,
        0.        ],
       [0.08581944, 1.        , 0.09805807, ..., 0.        , 0.        ,
        0.        ],
       [0.12155416, 0.09805807, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.3354102 ,
        0.16903085],
       [0.        , 0.        , 0.        , ..., 0.3354102 , 1.        ,
        0.09449112],
       [0.        , 0.        , 0.        , ..., 0.16903085, 0.09449112,
        1.        ]])

In [68]:
def recommend(product_id):
    index = new_df[new_df['product_id'] == product_id].index[0]
    distances = sorted(list(enumerate(similar[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:11]:
        print(new_df.iloc[i[0]].product_name)

In [69]:
recommend('B07JW9H4J1')

Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
Wayona Nylon Braided Usb Syncing And Charging Cable Sync And Charging Cable For Iphone, Ipad (3 Ft, Black) - Pack Of 2
Wayona Nylon Braided 3A Lightning to USB A Syncing and Fast Charging Data Cable for iPhone, Ipad (3 FT Pack of 1, Black)
Wayona Nylon Braided (2 Pack) Lightning Fast Usb Data Cable Fast Charger Cord For Iphone, Ipad Tablet (3 Ft Pack Of 2, Grey)
Wayona Usb Nylon Braided Data Sync And Charging Cable For Iphone, Ipad Tablet (Red, Black)
Wayona Nylon Braided 2M / 6Ft Fast Charge Usb To Lightning Data Sync And Charging Cable For Iphone, Ipad Tablet (6 Ft Pack Of 1, Grey)
Wayona Nylon Braided Lightning USB Data Sync & 3A Charging Cable for iPhones, iPad 

In [70]:
def link(product_name):
    index=df[new_df['product_name'] == product_name].index[0]
    productlink=df.iloc[index].product_link
    print(productlink)

In [72]:
def sentiment(product_name):
    index=df[new_df['product_name'] == product_name].index[0]
    title=df.iloc[index].review_title
    content=df.iloc[index].review_content
    print(title,content)

In [73]:
link('Wayona Usb Nylon Braided Data Sync And Charging Cable For Iphone, Ipad Tablet (Red, Black)')
sentiment("Wayona Usb Nylon Braided Data Sync And Charging Cable For Iphone, Ipad Tablet (Red, Black)")

https://www.amazon.in/Wayona-Braided-Syncing-Charging-iPhone/dp/B07LGT55SJ/ref=sr_1_89?qid=1672909128&s=electronics&sr=1-89
Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T2/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money


In [76]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json: 100%|██████████| 747/747 [00:00<?, ?B/s] 
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 1.01MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 711kB/s]
special_tokens_map.json: 100%|██████████| 150/150 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 499M/499M [02:22<00:00, 3.50MB/s] 
